In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from lr.train import train_lr
from lr.hyperparameters import BEST_HPS
import pandas as pd

## Download training data

The training data consists of about 600M whitespace tokens; 300M tokens from Common Crawl, 100M tokens from OpenWebText, Wikipedia, and Books3 corpus each. The test/dev data are 10M tokens each.

In [4]:
!mkdir -p filter_data
!curl -Lo filter_data/test.jsonl http://arkdata.cs.washington.edu/sg01/gpt3-filter/test.jsonl
!curl -Lo filter_data/dev.jsonl http://arkdata.cs.washington.edu/sg01/gpt3-filter/dev.jsonl
!curl -Lo filter_data/train.jsonl http://arkdata.cs.washington.edu/sg01/gpt3-filter/train.jsonl



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  367M  100  367M    0     0  27.0M      0  0:00:13  0:00:13 --:--:-- 30.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349M  100  349M    0     0  29.0M      0  0:00:12  0:00:12 --:--:-- 28.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2878M  100 2878M    0     0  31.9M      0  0:01:30  0:01:30 --:--:-- 49.9M.5M      0  0:04:09 --:--:--  0:04:09 11.5M    0  0:01:35  0:00:31  0:01:04 21.2M


## Read data

In [4]:
train = pd.read_json('filter_data/train.jsonl', lines=True)
dev = pd.read_json('filter_data/dev.jsonl', lines=True)
test = pd.read_json('filter_data/test.jsonl', lines=True)

## Train Classifer

(this will take about 10-15 minutes)

In [5]:
clf, vectorizer, results = train_lr(train, dev, test, BEST_HPS)

/homes/gws/sg01/quality-filter/lr/train.py:36: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  master = pd.concat([train, dev], 0)


In [11]:
results

,penalty,C,multi_class,solver,tol,random_state,dev_f1,test_f1,dev_accuracy,test_accuracy,training_duration,ngram_range,weight,stopwords
0,l1,0.977778,auto,liblinear,0.000816,44555,0.902953,0.904029,0.916117,0.917365,878.14413,"[1, 2]",hash,None


In [6]:
from data.score import score

In [18]:
wiki_text = """
Blakeney Chapel is a ruined building on the Norfolk coast of England. Even though named as such, it was probably not a chapel, and is not in the adjoining village of Blakeney, but rather in the parish of Cley next the Sea. The building stood on a raised mound or "eye" on the seaward end of the coastal marshes, less than 200 m (220 yd) from the sea and just to the north of the current channel of the River Glaven where it turns to run parallel to the shoreline. It consisted of two rectangular rooms of unequal size, and appears to be intact in a 1586 map, but is shown as ruins in later charts. Only the foundations and part of a wall still remain. Three archaeological investigations between 1998 and 2005 provided more detail of the construction, and showed two distinct periods of active use. Although it is described as a chapel on several maps, there is no documentary or archaeological evidence to suggest that it had any religious function. A small hearth, probably used for smelting iron, is the only evidence of a specific activity on the site.

Much of the structural material was long ago carried off for reuse in buildings in Cley and Blakeney. The surviving ruins are protected as a scheduled monument and Grade II listed building because of their historical importance, but there is no active management. The ever-present threat from the encroaching sea is likely to accelerate following a realignment of the Glaven's course through the marshes, and lead to the loss of the ruins.
"""
score(wiki_text, clf, vectorizer)

array([[0.21381688, 0.78618312]])

In [19]:
random_webtext = """
As someone who comes from a country with a lot of mountains and hills, I would highly recommend going trekking to places accessible only by foot. It's really nice to see the untouched,peaceful nature existing there, without humans to ruin it.
"""
score(random_webtext, clf, vectorizer)

array([[0.91200186, 0.08799814]])